In [1]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')

Using TensorFlow backend.


In [2]:
import numpy as np
seed = 7
np.random.seed(seed)

In [3]:
cifar10_dir = 'cs231n/datasets/cifar-10-batches-py'
from scipy.misc import imread

In [4]:
from six.moves import cPickle as pickle
import platform

In [5]:
def load_pickle(f):
    return pickle.load(f, encoding = 'latin1')

In [6]:
def load_CIFAR_batch(filename):
    with open(filename, 'rb') as f:
        datadict = load_pickle(f)
        X = datadict['data']
        Y = datadict['labels']
        X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
        Y = np.array(Y)
        return X, Y

In [7]:
def load_CIFAR10():
    xs = []
    ys = []
    root = 'C:\\Users\\Abarn\\Documents\\assignment1\\cs231n\\datasets\\cifar-10-batches-py'
    for b in range(1,6):
        f = os.path.join(root, 'data_batch_%d'%(b, ))
        X, Y = load_CIFAR_batch(f)
        xs.append(X)
        ys.append(Y)
    Xtr = np.concatenate(xs)
    Ytr = np.concatenate(ys)
    del X, Y
    Xte, Yte = load_CIFAR_batch(os.path.join(root,'test_batch'))
    return Xtr, Ytr, Xte, Yte

In [8]:
import os
X_train, y_train, X_test, y_test = load_CIFAR10()

In [9]:
print('Training data shape: ', X_train.shape)
print('Training labels shape: ', y_train.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train/255.0
X_test = X_test/255.0
print('Test data shape: ', X_test.shape)

Training data shape:  (50000, 32, 32, 3)
Training labels shape:  (50000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)
Test data shape:  (10000, 32, 32, 3)


In [10]:
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]
X_test.shape

(10000, 32, 32, 3)

In [11]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape = (3, 32, 32), padding = 'same', activation = 'relu', kernel_constraint = maxnorm(3)))
model.add(Dropout(0.2))
model.add(Conv2D(32, (3, 3), padding = 'same', activation = 'relu', kernel_constraint = maxnorm(3)))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dense(512, activation = 'relu', kernel_constraint = maxnorm(3)))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation = 'softmax'))

epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr = lrate, momentum = 0.9, decay = decay, nesterov = False)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])


In [13]:
print(model.summary())
X_test.shape

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 16, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               4194816   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
__________

(10000, 32, 32, 3)

In [14]:
print(X_train.shape)
X_train = X_train.transpose(0,3,1,2)
X_test = X_test.transpose(0,3,1,2)
print(X_train.shape)
model.fit(X_train, y_train, validation_data = (X_test, y_test), epochs = epochs, batch_size = 32 )
scores = model.evaluate(X_test, y_test, verbose = 0)
print("Accuracy:%.2f%%"%(scores[1]*100))

(50000, 32, 32, 3)
(50000, 3, 32, 32)
Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 116s 2ms/step - loss: 1.7238 - acc: 0.3745 - val_loss: 1.3949 - val_acc: 0.4972
Epoch 2/25
50000/50000 [==============================] - 109s 2ms/step - loss: 1.3594 - acc: 0.5112 - val_loss: 1.2498 - val_acc: 0.5601
Epoch 3/25
50000/50000 [==============================] - 110s 2ms/step - loss: 1.1913 - acc: 0.5755 - val_loss: 1.0919 - val_acc: 0.6110
Epoch 4/25
50000/50000 [==============================] - 109s 2ms/step - loss: 1.0620 - acc: 0.6240 - val_loss: 1.0479 - val_acc: 0.6289
Epoch 5/25
50000/50000 [==============================] - 107s 2ms/step - loss: 0.9563 - acc: 0.6581 - val_loss: 0.9692 - val_acc: 0.6595
Epoch 6/25
50000/50000 [==============================] - 106s 2ms/step - loss: 0.8730 - acc: 0.6941 - val_loss: 0.9477 - val_acc: 0.6644
Epoch 7/25
50000/50000 [==============================] - 107s 2ms/step - loss: 0.79

In [27]:
prediction = model.predict(X_test[1:10])
prediction = np.array([np.argmax(prediction[i]) for i in range(len(prediction))])
#prediction = np_utils.to_categorical(prediction)
print(prediction)

[8 8 0 6 6 1 6 3 1]


2

In [60]:
y_train.shape

(50000, 10)

In [61]:
y_test.shape

(10000, 10)